# 实验5
## PageRank

### 设置

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 数据加载

本实验中我们将使用Python工具包[NetworkX](https://networkx.github.io)。其主要功能是创建、操作复杂网络，并对网络的结构、动态和功能进行分析。

本实验中我们分析的数据是[斯坦福大学](https://stanford.edu)2002年的网络快照。节点代表斯坦福大学的网页，而有向边代表网页之间的超链接。 [[更多信息]](http://snap.stanford.edu/data/web-Stanford.html)

In [2]:
import networkx as nx

G = nx.read_edgelist('web-Stanford.txt', create_using=nx.DiGraph)

In [3]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 281903
Number of edges: 2312497
Average in degree:   8.2032
Average out degree:   8.2032


### 你的任务

图的连通分量是图的极大连通子图，连通分量内的任意两个结点之间都有直接或者间接的路径可达。
作为实验的开始，为了简化分析目标，我们先忽略原始网络图结构中的悬空结点（无法从该结点到达其他结点）和非连通分量。

使用NetworkX找出图```G```中**最大**的弱连通分量。我们将在接下来的实验中一直使用得到的连通分量（）。

In [4]:
largest_weakly_cc = max(nx.weakly_connected_components(G), key=len)
subgraph = G.subgraph(largest_weakly_cc)

使用NetworkX中的默认参数版本计算PageRank向量：[https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pageranky](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html#networkx.algorithms.link_analysis.pagerank_alg.pagerank)

In [5]:
page_rank = nx.pagerank(subgraph)
page_rank

{'1': 5.893078431557277e-07,
 '6548': 3.2918963109976826e-06,
 '15409': 3.2918963109976826e-06,
 '57031': 5.400258617197362e-06,
 '13102': 5.400258617197362e-06,
 '2': 0.00017105755205648759,
 '17794': 1.7643773942739463e-06,
 '25202': 1.7643773942739463e-06,
 '53625': 1.7643773942739463e-06,
 '54582': 1.7643773942739463e-06,
 '64930': 1.7643773942739463e-06,
 '73764': 1.7643773942739463e-06,
 '84477': 1.7643773942739463e-06,
 '98628': 1.7643773942739463e-06,
 '100193': 1.7643773942739463e-06,
 '102355': 1.7643773942739463e-06,
 '105318': 1.7643773942739463e-06,
 '105730': 1.7643773942739463e-06,
 '115926': 1.7643773942739463e-06,
 '140864': 1.7643773942739463e-06,
 '163550': 1.7643773942739463e-06,
 '164599': 1.7643773942739463e-06,
 '175799': 1.7643773942739463e-06,
 '178642': 1.7643773942739463e-06,
 '181714': 1.7643773942739463e-06,
 '190453': 1.7643773942739463e-06,
 '204189': 1.7643773942739463e-06,
 '204604': 1.7643773942739463e-06,
 '210870': 1.7643773942739463e-06,
 '213966': 

在1999年时，Barabási和Albert提出了一个优雅的数学模型，能够生成具有与Web图相似拓扑属性的图结构（也被称为尺度无关网络）。

如果你完成了下面的步骤，你应该能得到一些经验证据，表明在试图生成与万维网相似的图结构时，随机图模型的效果是劣于Barabási–Albert模型的。

按照这个思路，我们将使用两种不同的图生成方法，然后通过比较相应PageRank向量的方法测试它们与Web图的相似度。请参考[[NetworkX中的图生成器]](https://networkx.github.io/documentation/stable/reference/generators.html#)

将每种方法的随机种子都设置为```seed = 1```，生成：

1. 一个随机的图（选择fast方法），将```n```设置为等于原连通分量中的结点数量，以及```p = 0.00008```
2. 一个Barabasi-Albert图（选择标准方法），将```n```设置为等于原连通分量中的结点数量，并找到合适的**整数值**```m```，使得生成图中边的数量**刚好超过**原连通分量中的边数。

然后为每个图计算PageRank向量。

In [6]:
print(nx.info(subgraph))

Name: 
Type: DiGraph
Number of nodes: 255265
Number of edges: 2234572
Average in degree:   8.7539
Average out degree:   8.7539


In [7]:
n_nodes = len(largest_weakly_cc)
random_graph = nx.fast_gnp_random_graph(n_nodes, p=0.00008, seed=1)
barabasi_albert_graph = nx.barabasi_albert_graph(n_nodes, seed=1, m=9)
n_edges = barabasi_albert_graph.number_of_edges()
print(n_edges)

2297304


In [8]:
random_graph_pagerank = nx.pagerank(random_graph)
barabasi_albert_graph_pagerank = nx.pagerank(barabasi_albert_graph)

original_vec = np.array(sorted(page_rank.values()))
random_vec = np.array(sorted(random_graph_pagerank.values()))
barabasi_albert_vec = np.array(sorted(barabasi_albert_graph_pagerank.values()))

比较你在生成图上得到的PageRank向量与你在原始连通分量上得到的PageRank向量。
具体来说，你需要**排序**每个向量的所有分量，然后使用cosine相似度作为相似性的指标。

你可以使用任意第三方库的预先相似度实现方法。或者你也可以自己用```numpy```实现。

In [9]:
def cosim(a, b):
    inner_product = np.sum(a * b)
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    return inner_product / (a_norm * b_norm)

In [11]:
original_random_cosim = cosim(original_vec, random_vec)
original_brarbasialbert_cosim = cosim(original_vec, barabasi_albert_vec)
print('original_random_cosim = ',original_random_cosim)
print('original_brarbasialbert_cosim = ',original_brarbasialbert_cosim)

original_random_cosim =  0.10395564703964187
original_brarbasialbert_cosim =  0.6612451256426742
